In [7]:
import pandas as pd
import re
import json
from tqdm import tqdm

# Load data
df = pd.read_csv(r"C:\Users\user\Desktop\tripadvisor_hotel_reviews.csv")
df = df.dropna()

# Clean text
def clean_text(text):
    text = re.sub(r'<.*?>', '', text)  # remove HTML tags
    text = re.sub(r'[^a-zA-Z0-9\s.,!?]', '', text)  # remove special characters
    return text.strip()

df['clean_review'] = df['Review'].apply(clean_text)

# Split into ~500 token chunks (using simple sentence split)
chunks = []
chunk_id = 0

for i, row in tqdm(df.iterrows(), total=len(df)):
    sentences = re.split(r'(?<=[.!?]) +', row['clean_review'])  # Simple sentence split
    current_chunk = ""
    for sentence in sentences:
        if len(current_chunk.split()) + len(sentence.split()) < 500:
            current_chunk += " " + sentence
        else:
            chunks.append({
                "id": chunk_id,
                "source": "tripadvisor",
                "category": "hotel_review",
                "text": current_chunk.strip()
            })
            chunk_id += 1
            current_chunk = sentence
    if current_chunk:
        chunks.append({
            "id": chunk_id,
            "source": "tripadvisor",
            "category": "hotel_review",
            "text": current_chunk.strip()
        })
        chunk_id += 1

# Save to JSON
with open("processed_chunks.json", "w", encoding='utf-8') as f:
    json.dump(chunks, f, ensure_ascii=False, indent=2)

print("✅ Processing complete. Total chunks:", len(chunks))


100%|██████████| 20491/20491 [00:01<00:00, 15329.62it/s]


✅ Processing complete. Total chunks: 20748
